In [1]:
from uniswappy import *

user_nm = 'user0'
eth_amount = 10000
tkn_amount = 100000

fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [2]:
n_steps = 250
start_price = tkn_amount/eth_amount
mu = 0.3; sigma = 0.8
n_paths = 25

b = BrownianModel(start_price)
p_arr = b.gen_gbms(mu, sigma, n_steps, n_paths)
exp_p_arr = np.median(p_arr, axis = 1)

In [3]:
tkn = ERC20("TKN", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", address="0x011")

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, tkn_amount)
lp.summary()

Exchange ETH-TKN (LP)
Reserves: ETH = 10000, TKN = 100000
Liquidity: 31622.776601683792 



In [4]:
arb = CorrectReserves(lp, x0 = 1/exp_p_arr[0])
for k in range(1, n_steps):
    p = 1/exp_p_arr[k]
    arb.apply(p)

    select_tkn = EventSelectionModel().bi_select(0.5)
    bias_factor = 1
    rnd_amt = TokenDeltaModel(30).delta()
    
    if(select_tkn == 0):
        out = Swap().apply(lp, eth, user_nm, rnd_amt) 
    else:
        out = Swap().apply(lp, tkn, user_nm,  bias_factor*p*rnd_amt)  

    #print(f'price {lp.get_price(tkn)} {exp_p_arr[k]}')

print('')
lp.summary()


Exchange ETH-TKN (LP)
Reserves: ETH = 515244.9122660737, TKN = 50516.187167456206
Liquidity: 161047.06717309097 



In [5]:
eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, tkn_amount, lwr_tick, upr_tick)
lp.summary()

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 9999.999999999998, TKN = 100000.0
Liquidity: 31622.776601683792 



In [6]:
arb = CorrectReserves(lp, x0 = 1/exp_p_arr[0])
for k in range(1, n_steps):
    p = 1/exp_p_arr[k]
    arb.apply(p, lwr_tick, upr_tick)

    select_tkn = EventSelectionModel().bi_select(0.5)
    bias_factor = 1
    rnd_amt = TokenDeltaModel(100).delta()
    
    if(select_tkn == 0):
        out = Swap().apply(lp, eth, user_nm, rnd_amt) 
    else:
        out = Swap().apply(lp, tkn, user_nm,  bias_factor*p*rnd_amt)  

    #print(f'price {lp.get_price(tkn)} {exp_p_arr[k]}')

print('')
lp.summary()


Exchange ETH-TKN (LP)
Real Reserves:   ETH = 515047.75886891154, TKN = 50471.51255559729
Liquidity: 161124.6166287103 

